# Práctica 4

**Nombre:** Anahí Barajas Pérez  
**e-mail:** anahi.barajas2412@alumnos.udg.mx

## MODULES

In [ ]:
import panel as pn
import panel.widgets as pnw
import holoviews as hv
from holoviews import opts
pn.extension(sizing_mode = 'stretch_width')
hv.extension('plotly')

#pn.extension('plotly',raw_css=[css])

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from pymunk import Vec2d


## FUNCTIONS

In [ ]:
def bm_2d(n_steps=1000,speed=6,s_x_pos=0,s_y_pos=0):
  '''
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  '''
  #Init Vector
  velocity = Vec2d(speed,0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df],ignore_index=True)

  for i in range(n_steps - 1):
    turn_angle = np.random.uniform(low=-np.pi,high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index = True)

  #Return dataframe
  return BM_2d_df

In [ ]:
radioTrajType = pnw.RadioButtonGroup(
    name='Trajectorie Type', options=['BM', 'CRW', 'LF'], button_type='primary')

@pn.depends(radioTrajType)
def display_RadioTraj(radioTrajType):
  return radioTrajType

stepSlider = pnw.IntSlider(name='Number of steps', value = 1000, start=100, end=5000, step = 100)

@pn.depends(stepSlider)
def display_StepSlider(stepSlider):
  return stepSlider

xInput = pnw.IntInput(name = 'X intial position', value = 0, step=1, start=0, end=200)

@pn.depends(xInput)
def display_xInput(xInput):
  return xInput

yInput = pnw.IntInput(name = 'Y intial position', value = 0, step=1, start=0, end=200)

@pn.depends(yInput)
def display_yInput(yInput):
  return yInput

speedSlider = pnw.IntSlider(name='Speed', value = 3, step = 1, start = 1, end = 20, sizing_mode='stretch_width')

@pn.depends(speedSlider)
def display_SpeedSlider(speedSlider):
  return speedSlider

metricSelect = pnw.Select(name='Metric Type', options=['MSD','PL'], sizing_mode='stretch_width')

@pn.depends(metricSelect)
def display_MetricSelect(metricSelect):
  return metricSelect

In [ ]:
@pn.depends(stepSlider=stepSlider, speedSlider=speedSlider, xInput=xInput, yInput=yInput)
def plot_traj(stepSlider, speedSlider, xInput, yInput):
        
    rw_df = bm_2d(n_steps=stepSlider, speed=speedSlider, s_x_pos=xInput, s_y_pos=yInput)
    
    fig_traj_rw = go.Figure(layout=dict(title='Browninan Motion trajectory'+f'steps = {stepSlider}', height=600))
    fig_traj_rw.add_trace(
        go.Scatter3d(
            x=rw_df.x_pos,
            y=rw_df.y_pos,
            z=rw_df.index,
            line_width=2,
            line_color='#D19D01',
            mode = 'lines',
            name = f'steps = {stepSlider}'
        )
    )
    return fig_traj_rw

In [ ]:
bootstrap = pn.template.BootstrapTemplate(title='Random Walk Metrics: Path Length/MSD'
                                          ,sidebar_width = 220
                                          ,header_background='#00504C')
logo = 'https://panel.holoviz.org/_static/logo_horizontal.png'

bootstrap.sidebar.append(radioTrajType)
bootstrap.sidebar.append(stepSlider)
bootstrap.sidebar.append(xInput)
bootstrap.sidebar.append(yInput)
bootstrap.sidebar.append(speedSlider)
bootstrap.sidebar.append(metricSelect)


bootstrap.main.append(
    pn.Row(
        pn.Card(plot_traj, title='Plot Trajectory'),
        pn.Card(plot_traj, title='Plot Trajectory')
    )
)

bootstrap.show();